In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
import pandas as pd
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
prevmodel = "gstock-4b-test"
model = "gstock-6b-test"
tickers = [ 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
prevmodel = "val-5a-test"
model = "val-7a-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
prevmodel = "cry-5a-test"
model = "cry-7a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']

In [11]:
findata.EPOCHS=200
prevmodel = "etf-5b-test"
model = "etf-7b-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-5b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS', 'DELL',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [10]:
pd.options.display.max_columns = None
addAllocHL(df, 0.10, 1)
df1 = df[df.Gain > 0]
df1.sort_values('Alloc', ascending=False).head(50)
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index]
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
72,TTD,1.50,0.98,799.75,707.06,96.01,100.04,0.04,1.13,0.60,489.75,421.27,104.37,0.09,0.98,0.74,972.71,745.24,95.28,-0.01,0.04,9.300000
6,BILL,4.15,0.93,1770.63,1854.99,53.61,68.19,0.27,3.91,0.69,1437.91,1555.08,78.30,0.46,2.52,0.78,1715.40,1537.86,52.82,-0.01,0.27,9.040741
74,U,2.35,0.92,708.00,988.61,16.21,24.01,0.48,1.92,0.70,611.78,1153.89,21.98,0.36,1.51,0.72,507.77,921.42,16.02,-0.01,0.36,8.977778
3,AMBA,2.39,0.93,972.70,919.69,56.74,67.37,0.19,1.98,0.69,894.79,1011.17,70.99,0.25,1.56,0.79,884.34,1129.81,54.34,-0.04,0.19,8.931579
0,ABNB,2.23,0.95,962.20,965.92,147.99,162.98,0.10,1.76,0.76,1088.53,1041.54,159.71,0.08,1.40,0.74,913.96,1113.80,140.57,-0.05,0.08,8.875000
16,DDOG,1.99,0.94,995.79,1185.88,119.03,131.20,0.10,1.73,0.80,1192.27,1159.66,144.66,0.22,1.39,0.73,943.48,924.67,116.79,-0.02,0.10,8.800000
17,DOCN,1.39,0.92,452.65,591.10,32.82,38.70,0.18,1.07,0.76,439.75,474.29,43.33,0.32,0.90,0.77,543.96,404.89,33.23,0.01,0.18,8.755556
41,MNDY,4.93,0.94,1093.27,1958.20,228.65,250.16,0.09,3.80,0.78,1790.81,1725.73,264.74,0.16,3.25,0.76,1571.20,1565.39,204.80,-0.10,0.09,8.733333
64,SNOW,4.16,0.92,1674.50,2018.97,129.85,151.29,0.17,3.30,0.74,1599.22,1998.43,151.59,0.17,3.26,0.77,1809.28,2209.77,130.18,0.00,0.17,8.729412
9,CFLT,1.11,0.92,254.03,344.08,24.37,28.51,0.17,0.87,0.82,407.90,490.52,30.75,0.26,0.67,0.79,308.23,410.16,24.55,0.01,0.17,8.729412


In [ ]:
findata.EPOCHS=200
model = "vols-6b-test"
pending = ['ASTR', 'SMWB', 'CZOO', 'DMTK']
next = []
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [7]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-6a-test"
next = []
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-6a-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-6a-test"
tickers = [ 'FXE', 'FXY', 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
dfprev  = pipeline.runModelCombinedVola(tickers, prevmodel, mod, False, loss=lossfn)

In [13]:
addAllocHL(dfprev, 0.10, 1)
df1prev = dfprev[dfprev.Gain > 0]
pd.options.display.max_columns = None
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
df1prev.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
11,IPO,0.62,0.93,227.42,280.58,41.25,46.74,0.13,0.68,0.77,236.36,268.43,46.42,0.13,0.44,0.64,214.00,248.61,39.54,-0.04,0.13,8.761538
22,ONLN,0.88,0.92,210.40,357.83,40.50,48.98,0.21,0.67,0.66,102.06,208.12,45.90,0.13,0.59,0.68,193.29,311.71,38.89,-0.04,0.13,8.584615
33,XITK,2.31,0.94,769.65,1083.42,148.75,158.17,0.06,1.77,0.78,792.38,735.95,159.23,0.07,1.51,0.73,857.87,1007.87,145.15,-0.02,0.06,8.400000
1,ARKK,1.70,0.93,413.38,631.43,46.63,49.52,0.06,1.20,0.66,130.48,305.22,50.06,0.07,0.88,0.65,462.47,518.02,43.71,-0.06,0.06,8.133333
2,ARKW,1.80,0.90,547.21,569.06,81.50,88.18,0.08,1.44,0.76,449.59,510.67,87.76,0.08,1.12,0.64,545.33,544.81,76.36,-0.06,0.08,7.750000
0,ARKF,0.64,0.91,262.49,171.24,28.69,30.98,0.08,0.50,0.76,157.60,166.95,30.14,0.05,0.38,0.67,178.08,149.74,27.36,-0.05,0.05,7.300000
17,IYZ,0.17,0.92,71.73,69.32,22.30,23.19,0.04,0.12,0.71,62.08,58.01,23.33,0.05,0.12,0.62,56.32,53.05,21.41,-0.04,0.04,7.200000
18,JETS,0.29,0.88,108.50,98.28,19.11,20.44,0.07,0.20,0.71,121.63,79.87,21.20,0.11,0.17,0.63,101.89,92.63,18.50,-0.03,0.07,7.085714
9,ICLN,0.44,0.86,117.95,107.01,13.67,14.46,0.06,0.34,0.72,92.14,79.44,14.64,0.07,0.32,0.77,81.46,119.37,13.51,-0.01,0.06,6.266667
46,XNTK,3.75,0.87,672.87,454.86,188.68,198.03,0.05,2.83,0.63,415.08,141.83,199.05,0.05,2.35,0.61,376.30,149.77,188.71,0.00,0.05,6.100000


In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [15]:
addAllocHL(df, 0.10, 1)
df1 = df[df.Gain > 0]
pd.options.display.max_columns = None
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
# df1.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
4,DAPP,0.33,0.93,117.40,116.94,13.90,12.79,-0.08,0.29,0.66,42.83,66.72,15.53,0.12,0.22,0.70,84.61,78.89,12.28,-0.12,-0.08,8.425000
2,ARKW,1.48,0.95,654.29,697.46,81.50,87.50,0.07,1.20,0.77,473.15,595.86,87.70,0.08,1.19,0.70,578.97,728.61,79.59,-0.02,0.07,8.785714
0,ARKF,0.49,0.93,247.29,180.66,28.69,30.74,0.07,0.44,0.69,107.95,139.49,31.66,0.10,0.45,0.81,196.52,306.56,28.08,-0.02,0.07,8.300000
46,XNTK,1.26,0.96,780.49,528.50,188.68,199.17,0.06,1.00,0.69,498.58,426.52,202.00,0.07,1.00,0.59,588.74,284.90,188.53,-0.00,0.06,8.933333
24,SMH,2.20,0.90,948.96,581.75,248.14,261.99,0.06,1.65,0.68,708.42,403.32,267.48,0.08,1.56,0.73,833.45,383.76,250.61,0.01,0.06,7.333333
9,ICLN,0.26,0.93,132.09,109.97,13.67,14.38,0.05,0.18,0.63,55.40,97.29,14.83,0.08,0.20,0.79,94.79,116.25,13.81,0.01,0.05,7.900000
13,IXN,0.53,0.93,293.05,162.75,81.24,85.52,0.05,0.38,0.82,343.93,136.58,87.20,0.07,0.42,0.80,290.13,202.61,79.83,-0.02,0.05,7.900000
22,ONLN,0.79,0.87,200.94,317.86,40.50,43.50,0.07,0.54,0.70,181.39,235.69,43.06,0.06,0.61,0.76,222.20,315.82,40.60,0.00,0.06,6.533333
15,IWM,1.84,0.91,792.29,569.35,216.84,206.92,-0.05,1.07,0.79,974.95,714.92,217.84,0.00,1.11,0.70,798.42,496.41,206.97,-0.05,-0.05,7.300000
18,JETS,0.27,0.93,113.35,111.44,19.11,19.85,0.04,0.18,0.72,97.75,116.16,20.78,0.09,0.19,0.67,104.69,69.42,19.24,0.01,0.04,7.550000


In [ ]:
findata.EPOCHS=200
model = "sbee-626-test"
tickersstr = "MLGO SLNH MDIA TBLT ALIM LICY LTBR ALNY HUT WULF SMR " + \
"RIVN EGRX BZFD XPOF ENVX SDIG BTOG APLD CORZ AILE CHWY SRPT BTBT AOSL " + \
"RNA RXO MINM CVRX ICU SISI NRIX MIRM OCUL CMTL DAKT"
tickers = tickersstr.split()
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


dfsbee  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)


In [ ]:
addAllocHL(dfsbee, 0.10, 1)
pd.options.display.max_columns = None
dfsbee.iloc[(dfsbee.Alloc * abs(dfsbee.Gain_f)).sort_values(ascending=False).index].head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,TBLT,309.16,0.66,27839.01,74331.96,5.18,185.05,34.72,222.49,0.44,46723.60,49875.36,314.87,59.79,189.35,0.41,30957.97,47579.28,191.92,36.05,36.05,6.590569
1,SLNH,7.76,0.73,1564.53,1885.61,6.01,9.19,0.53,6.52,0.55,2235.57,2207.76,12.15,1.02,5.25,0.52,1728.95,1176.65,5.17,-0.14,0.53,6.790566
0,MLGO,1.84,0.70,196.93,489.47,19.74,9.55,-0.52,6.55,0.48,1310.99,1436.11,47.14,1.39,2.03,0.48,132.73,427.73,9.54,-0.52,-0.52,6.423077
16,SDIG,3.85,0.76,248.60,558.57,4.37,6.09,0.39,3.19,0.47,-7.19,279.28,8.18,0.87,2.74,0.43,27.06,409.95,4.96,0.14,0.39,6.984615
8,HUT,1.68,0.82,430.66,386.17,15.30,10.07,-0.34,1.16,0.62,328.03,352.10,15.56,0.02,0.85,0.79,447.87,342.70,9.46,-0.38,-0.34,7.670588
31,NRIX,1.04,0.87,225.69,250.02,20.75,28.49,0.37,0.96,0.63,204.44,234.27,26.37,0.27,0.57,0.67,164.34,131.92,17.83,-0.14,0.27,8.218519
9,WULF,0.55,0.79,113.11,96.40,4.46,3.34,-0.25,0.55,0.76,175.59,149.81,4.80,0.08,0.37,0.62,82.69,112.71,2.39,-0.46,-0.25,7.060000
10,SMR,0.32,0.85,30.80,44.61,11.70,9.29,-0.21,0.23,0.66,51.14,46.16,10.60,-0.09,0.19,0.74,23.81,34.43,6.18,-0.47,-0.21,7.785714
7,ALNY,4.05,0.89,1192.44,1345.74,246.93,190.94,-0.23,3.04,0.79,1356.41,995.88,234.77,-0.05,2.00,0.71,974.80,1129.56,199.25,-0.19,-0.19,8.321053
14,XPOF,0.83,0.86,127.68,149.02,15.34,18.47,0.20,0.53,0.79,187.84,170.49,27.93,0.82,0.45,0.70,135.95,103.76,13.58,-0.11,0.20,7.900000


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [16]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
etf-7b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-7b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-7b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,649,002 bytes  received 73 bytes  19,298,150.00 bytes/sec
total size is 227,996,248  speedup is 23.63
